#### Setup environment
1. download this model, link: https://github.com/Jiaxin-Ye/TIM-Net_SER.git
2. put this file into TIM-Net_SER/Code/

In [7]:
import numpy as np
from numpy import newaxis
import tensorflow as tf
# import tensorflow.keras.backend as K
import keras
from keras import backend as K
import os
from tensorflow.keras.optimizers import SGD,Adam
from tensorflow.keras import callbacks
from tensorflow.keras.layers import Layer,Dense,Input
from tensorflow.keras.models import Model
from sklearn.metrics import confusion_matrix
from Common_Model import Common_Model
from sklearn.model_selection import KFold
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
import datetime
import pandas as pd

from scipy.io import wavfile as sciwav
from scipy import signal
import librosa.display

from TIMNET import TIMNET

In [8]:
import os
import tensorflow as tf
import matplotlib.pyplot as plt
from Model import TIMNET_Model
import argparse

In [9]:
def load_audiofiles(path):
    all_audio_files = os.listdir(path)
    audio_files = [ fname for fname in all_audio_files if fname.endswith('.wav')]
    audio_files.sort()
    print(audio_files[:10])
    print(len(audio_files))
    return audio_files

In [24]:
audiopath = '/media/nali/Seagate Portable Drive/8_HRIStudy2_audioData_participantonly/part2_A1_B/wav/'
audiofiles = load_audiofiles(audiopath)

['11_face_B1_BA1_trim_user.wav', '11_face_B2_BA1_trim_user.wav', '11_face_B3_BA1_trim_user.wav', '11_face_B4_BA1_trim_user.wav', '12_face_B1_BA1_trim_user.wav', '12_face_B2_BA1_trim_user.wav', '12_face_B3_BA1_trim_user.wav', '12_face_B4_BA1_trim_user.wav', '13_face_B1_BA1_trim_user.wav', '13_face_B2_BA1_trim_user.wav']
157


In [25]:
#get mean of signal length
# A1_beforehelp
s_len_sum = 0
for audiofile in audiofiles:
    signal, fs = librosa.load(audiopath + audiofile)
    s_len = len(signal)
    s_len_sum += s_len
mean_signal_length = int(s_len_sum/len(audiofiles))
mean_signal_length

38146

In [91]:
#get MMCCs features --- test
# signal, fs = librosa.load(audiopath + audiofiles[1])
# s_len = len(signal)

# if s_len < mean_signal_length:
#     pad_len = mean_signal_length - s_len
#     pad_rem = pad_len % 2
#     pad_len //= 2
#     signal = np.pad(signal, (pad_len, pad_len + pad_rem), 'constant', constant_values = 0)
# else:
#     pad_len = s_len - mean_signal_length
#     pad_len //= 2
#     signal = signal[pad_len:pad_len + mean_signal_length]
# mfcc = librosa.feature.mfcc(y=signal, sr=fs, n_mfcc=39)

# feature = np.repeat(mfcc[:, :, np.newaxis], 10, axis=2)
# feature = feature.T
# mfcc_feature.append([mfcc, audiofiles[1]])

In [26]:
#get MMCCs features
mfcc_feature = [] 
for audiofile in audiofiles:
    
    signal, fs = librosa.load(audiopath + audiofile)
    s_len = len(signal)

    if s_len < mean_signal_length:
        pad_len = mean_signal_length - s_len
        pad_rem = pad_len % 2
        pad_len //= 2
        signal = np.pad(signal, (pad_len, pad_len + pad_rem), 'constant', constant_values = 0)
    else:
        pad_len = s_len - mean_signal_length
        pad_len //= 2
        signal = signal[pad_len:pad_len + mean_signal_length]
    mfcc = librosa.feature.mfcc(y=signal, sr=fs, n_mfcc=39)

    feature = np.repeat(mfcc[:, :, np.newaxis], 10, axis=2)
    feature = feature.T
    mfcc_feature.append([feature, audiofile])

In [27]:
# mfcc_feature_all = {}GoDiS
mfcc_feature_all =  pd.DataFrame(mfcc_feature, columns = ['feature','filename'])

In [28]:
mfcc_feature_all.head()

,feature,filename
0,"[[[-797.5734, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0....",11_face_B1_BA1_trim_user.wav
1,"[[[-782.4599, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0....",11_face_B2_BA1_trim_user.wav
2,"[[[-425.25153, 121.934296, 12.968132, 34.21401...",11_face_B3_BA1_trim_user.wav
3,"[[[-461.3756, 93.42343, -12.372622, 31.628988,...",11_face_B4_BA1_trim_user.wav
4,"[[[-862.37775, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0...",12_face_B1_BA1_trim_user.wav


In [29]:
mfcc_feature_all.to_csv("/home/nali/Develop/HRI/data/HRI_Study2_result_csv/participantSpeech/speechEmotionDetection/emotionspeech_features_part2_A1_B.csv")

In [ ]:
plt.figure(figsize=(12,8))
librosa.display.specshow(mfcc, sr=librosa_sr,x_axis='time')
plt.colorbar(format='%+2.0f db')
plt.show()

In [144]:
array_loaded = np.load('/home/nali/Develop/HRI/SER MFCC Features/IEMOCAP.npy', allow_pickle=True)

In [118]:
path = './Test_Models/IEMOCAP_16'

In [17]:
SAVEE_CLASS_LABELS = ("angry","disgust", "fear", "happy", "neutral", "sad", "surprise")#SAVEE

In [137]:
IEMOCAP_CLASS_LABELS = ("angry", "happy", "neutral", "sad")

In [18]:
RAVDE_CLASS_LABELS = ("angry", "calm", "disgust", "fear", "happy", "neutral","sad","surprise")#rav

In [145]:
data = np.load("/home/nali/Develop/HRI/SER MFCC Features/IEMOCAP.npy",allow_pickle=True).item()
x_source_sample = data["x"]
y_source_sample = data["y"]

In [ ]:
# x_source = feature[newaxis, :, :]

In [146]:
feature.shape

(10, 107, 39)

In [147]:
x_source = feature

In [ ]:
# x_new = np.repeat(mfcc[:, :, np.newaxis], 10, axis=2)
# x_source = x_new.T

In [ ]:
# model = TIMNET_Model(input_shape=x_source.shape[1:], class_label=IEMOCAP_CLASS_LABELS)

In [ ]:
# x_feats, y_labels = model.test(x_source, y_source, path=path)# x_feats and y_labels are test datas for t-sne

In [148]:
class_label =IEMOCAP_CLASS_LABELS
num_classes = len(class_label)

In [149]:
x_source.shape

(10, 107, 39)

In [150]:
x_source.shape[1:][1]

39

In [126]:
for index, feature in mfcc_feature_all.iterrows():
    print(feature['filename'], index)
    break

10_face_B1_BA2_trim_user.wav 0


In [151]:
y_source = np.zeros((10, 4), dtype=int)
y_source.shape

(10, 4)

### Predicted results

In [ ]:
os.environ['CUDA_VISIBLE_DEVICES'] = "0"
gpus = tf.config.experimental.list_physical_devices(device_type='GPU')
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth=True 
session = tf.compat.v1.Session(config=config)
print(f"###gpus:{gpus}")

In [ ]:
def create_model(data_shape):
    inputs=Input(shape = (data_shape[0],data_shape[1]))
    multi_decision = TIMNET(nb_filters=39,
                            kernel_size=2, 
                            nb_stacks=1,
                            dilations=10, #if iemocap -> 10
                            dropout_rate=0.1,
                            activation = 'relu',
                            return_sequences=True, 
                            name='TIMNET')(inputs)

    decision = WeightLayer()(multi_decision)
    predictions = Dense(num_classes, activation='softmax')(decision)
    model = Model(inputs = inputs, outputs = predictions)

    model.compile(loss = "categorical_crossentropy",
                       optimizer =Adam(learning_rate=0.001, beta_1=0.93, beta_2=0.98, epsilon=1e-8),
                       metrics = ['accuracy'])
    # print("Temporal create succes!")
    return model

In [ ]:
def smooth_labels(labels, factor=0.1):
    # smooth the labels
    labels *= (1 - factor)
    labels += (factor / labels.shape[1])
    return labels

class WeightLayer(Layer):
    def __init__(self, **kwargs):
        super(WeightLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        self.kernel = self.add_weight(name='kernel',
                                      shape=(input_shape[1],1),
                                      initializer='uniform',
                                      trainable=True)  
        super(WeightLayer, self).build(input_shape)  
 
    def call(self, x):
        tempx = tf.transpose(x,[0,2,1])
        x = K.dot(tempx,self.kernel)
        x = tf.squeeze(x,axis=-1)
        return  x
 
    def compute_output_shape(self, input_shape):
        return (input_shape[0],input_shape[2])
    
def softmax(x, axis=-1):
    ex = K.exp(x - K.max(x, axis=axis, keepdims=True))
    return ex/K.sum(ex, axis=axis, keepdims=True)

In [ ]:
results = []
for index, feature in mfcc_feature_all.iterrows():
    x_source = feature['feature']
    i=0
    kfold = KFold(n_splits=10, shuffle=True, random_state=16) #SAVEE random_seed 44 #imocap 16
    avg_accuracy = 0
    avg_loss = 0
    x_feats = []
    y_labels = []
    matrix = []
    x = x_source
    y = y_source
    y_pred_best_all = []
    eva_matrix = []
    y_best_index = 0

    for i, (train_index, test_index) in enumerate(kfold.split(x)):
        print("hdf5 file index",i)
        model = create_model(x.shape[1:])
        weight_path=path+'/'+str(10)+"-fold_weights_best_"+str(i+1)+".hdf5"
        # model.fit(x[train], y[train],validation_data=(x[test],  y[test]),batch_size = 64,epochs = 0,verbose=0)
        model.fit(x_source_sample[train_index], y_source_sample[train_index],batch_size = 64,epochs = 0,verbose=0)
        model.load_weights(weight_path)#+source_name+'_single_best.hdf5')
        # best_eva_list = model.evaluate(x[test],  y[test])
        #avg_loss += best_eva_list[0]
        # avg_accuracy += best_eva_list[1]
        # print(str(i)+'_Model evaluation: ', best_eva_list,"   Now ACC:",str(round(avg_accuracy*10000)/100/i))
        i+=1
        y_pred_best = model.predict(x[test_index])
        y_pred_best_all.append(y_pred_best)
        y_best_index += np.argmax(y_pred_best)
        # matrix.append(confusion_matrix(np.argmax(y[test],axis=1),np.argmax(y_pred_best,axis=1)))
        # em = classification_report(np.argmax(y[test],axis=1),np.argmax(y_pred_best,axis=1), target_names=class_label,output_dict=True)
        # eva_matrix.append(em)
        # print(classification_report(np.argmax(y[test],axis=1),np.argmax(y_pred_best,axis=1), target_names=class_label))
        # caps_layer_model = Model(inputs=model.input,
        # outputs=model.get_layer(index=-2).output)
        # feature_source = caps_layer_model.predict(x[test])
        # x_feats.append(feature_source)
        # y_labels.append(y[test])
    #print("Average ACC:",avg_accuracy/10)
    #acc = avg_accuracy/10
    y_bast_index_best = y_best_index/10
    results.append([y_bast_index_best, feature['filename']])
    print("wavfile:",index)
print("done", index)

In [153]:
emotion_results =  pd.DataFrame(results, columns = ['scores','filename'])
emotion_results.head()

,scores,filename
0,0.1,10_face_B1_BA2_trim_user.wav
1,0.4,10_face_B2_BA2_trim_user.wav
2,0.3,10_face_B3_BA2_trim_user.wav
3,0.0,10_face_B4_BA2_trim_user.wav
4,0.6,14_face_B1_BA2_trim_user.wav


In [154]:
emotion_results.to_csv("/home/nali/Develop/HRI/data/HRI_Study2_result_csv/participantSpeech/SpeechEmotionDetection/emotionspeech_results_part2_A2_B.csv")

In [ ]:
y_pred_best_all

In [ ]:
np.argmax(y_pred_best_all[9],axis=1)

In [ ]:
np.argmax(y_pred_best_all[9])

In [ ]:
if np.argmax(y_pred_best_all[0]) in (0, 1, 2, 3):
                categoryscore = 0
            elif np.argmax(scores) in (3, 6):
                categoryscore = 1
            else:
                categoryscore = 2
            df_emotion_frame.loc[index] = [fname, np.argmax(scores), categoryscore]

In [ ]:
np.argmax(y_pred_best_all[0],axis=1)

In [ ]:
np.argmax(y_pred_best_all[1],axis=1)

In [ ]:
# model = TIMNET_Model(input_shape=x_source.shape[1:], class_label=IEMOCAP_CLASS_LABELS)

### Original code

In [ ]:
class TIMNET_Model(Common_Model):
    def __init__(self, input_shape, class_label, **params):
        super(TIMNET_Model,self).__init__(**params)
        # self.args = args
        self.data_shape = input_shape
        self.num_classes = len(class_label)
        self.class_label = class_label
        self.matrix = []
        self.eva_matrix = []
        self.acc = 0
        print("TIMNET MODEL SHAPE:",input_shape)
    
    def create_model(self):
        self.inputs=Input(shape = (self.data_shape[0],self.data_shape[1]))
        self.multi_decision = TIMNET(nb_filters=39,
                                kernel_size=2, 
                                nb_stacks=1,
                                dilations=10, #if iemocap -> 10
                                dropout_rate=0.1,
                                activation = 'relu',
                                return_sequences=True, 
                                name='TIMNET')(self.inputs)

        self.decision = WeightLayer()(self.multi_decision)
        self.predictions = Dense(self.num_classes, activation='softmax')(self.decision)
        self.model = Model(inputs = self.inputs, outputs = self.predictions)
        
        self.model.compile(loss = "categorical_crossentropy",
                           optimizer =Adam(learning_rate=0.001, beta_1=0.93, beta_2=0.98, epsilon=1e-8),
                           metrics = ['accuracy'])
        print("Temporal create succes!")
        return self.model
    
    def test(self, x, y, path):
        i=1
        kfold = KFold(n_splits=10, shuffle=True, random_state=44) #SAVEE random_seed 44
        avg_accuracy = 0
        avg_loss = 0
        x_feats = []
        y_labels = []
        for train, test in kfold.split(x, y):
            self.create_model()
            weight_path=path+'/'+str(10)+"-fold_weights_best_"+str(i)+".hdf5"
            self.model.fit(x[train], y[train],validation_data=(x[test],  y[test]),batch_size = 64,epochs = 0,verbose=0)
            self.model.load_weights(weight_path)#+source_name+'_single_best.hdf5')
            best_eva_list = self.model.evaluate(x[test],  y[test])
            avg_loss += best_eva_list[0]
            avg_accuracy += best_eva_list[1]
            print(str(i)+'_Model evaluation: ', best_eva_list,"   Now ACC:",str(round(avg_accuracy*10000)/100/i))
            i+=1
            y_pred_best = self.model.predict(x[test])
            self.matrix.append(confusion_matrix(np.argmax(y[test],axis=1),np.argmax(y_pred_best,axis=1)))
            em = classification_report(np.argmax(y[test],axis=1),np.argmax(y_pred_best,axis=1), target_names=self.class_label,output_dict=True)
            self.eva_matrix.append(em)
            print(classification_report(np.argmax(y[test],axis=1),np.argmax(y_pred_best,axis=1), target_names=self.class_label))
            caps_layer_model = Model(inputs=self.model.input,
            outputs=self.model.get_layer(index=-2).output)
            feature_source = caps_layer_model.predict(x[test])
            x_feats.append(feature_source)
            y_labels.append(y[test])
        print("Average ACC:",avg_accuracy/10)
        self.acc = avg_accuracy/10
        return x_feats, y_labels 